In [ ]:
import xarray as xr
import os, sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
module_path = '/home/jupyter-ayoub'
if module_path not in sys.path:
    sys.path.append(module_path)
from Transformer import StackedTransformer

In [ ]:
#ssrd6
test_rfcs=xr.open_dataset("/home/jupyter-ayoub/data/test/test_reforecast.nc")
obs=xr.open_dataset("/home/jupyter-ayoub/data/test/test_observation.nc")

In [ ]:
test_rfcs=test_rfcs
obs=obs

In [ ]:
target='ssrd6_obs'  
obs=obs[target]
test_rfcs=test_rfcs.drop_vars(['sd', 'mx2t6', 'mn2t6', 'q', 'ssr6', 'str6'])
#test_rfcs=test_rfcs.drop_vars(['t2m', 'z', 't', 'u10', 'v10', 'tcc', 'sd', 'mx2t6', 'mn2t6', 'w10', 'u100', 'w100', 'u', 'w700', 'p10fg6', 'v100', 'v', 'q', 'tp6', 'ssr6', 'str6', 'strd6'])
#test_rfcs=test_rfcs.drop_vars(['u10', 'v10', 'sd', 'mx2t6', 'mn2t6', 'w10', 'u100', 'w100', 'u', 'w700', 'p10fg6', 'v100', 'v', 'q', 'ssr6', 'str6']) #8
#[ 'sd', 'mx2t6', 'mn2t6', 'q', 'ssr6', 'str6']
#['sd', 'mx2t6', 'mn2t6', 'q', 'tp6', 'ssr6', 'str6', 'ssrd6', 'strd6']

In [ ]:
#add orography as a variable
oro=xr.open_dataset("/home/jupyter-aaron/Postprocessing/PP_EUPP/data/oro.nc") 
dims={'time' : test_rfcs['time'],
       'number': test_rfcs['number'],
       'step': test_rfcs['step']}
xds_oro=oro.expand_dims(time=dims['time'], number=dims['number'],step=dims['step'])
dummy_oro=xr.Dataset(coords=dims)
final_oro=xds_oro.broadcast_like(dummy_oro)

In [ ]:
fcs_10=xr.merge([test_rfcs,final_oro])
dummy=xr.merge([test_rfcs,final_oro])

In [ ]:
#normalizing met oro.nc
min_max_values={"t2m":(235, 304), "z": (48200, 58000), "t":(240, 299), "u10": (-13., 11.),"v10": (-30,35), "tcc": (0., 1.0),"sd":(0,8),"mx2t6":(230,320),"mn2t6":(225,315),"v":(-50,55), "w100":(0,50),"w10":(0,30), "u100": (-35,45), "u": (-45,60),"v100":(-40,45), "w700": (0,60), "p10fg6": (0,60), "oro":(-400,2800),"ssr6":(0,3200000),"ssrd6":(-5200.0,18541902.0),"strd6":(2103856,9537712),"tp6":(-4,1),"ssr6":(-4220.0,15344816.0),"str6":(-4630656.0,1093968.0)}
# Function to normalize using vectorized operations
def minmax_normalize(ds, min_max_values):
    for var in ds.data_vars:
        if var in min_max_values:
            min_val, max_val = min_max_values[var]
            ds[var] = (ds[var] - min_val) / (max_val - min_val)
    return ds

def z_normalize(ds):
    for var in ds.data_vars:
        ds[var] = ds[var].copy()  # Ensure the variable is writable
        
        for t in range(len(ds["time"])):  # Loop over time dimension
            mean = ds[var].isel(time=t).mean()
            std = ds[var].isel(time=t).std()
            ds[var].loc[dict(time=ds["time"][t])] = (ds[var].isel(time=t) - mean) / std

    return ds



dummy_norm=minmax_normalize(dummy,min_max_values)
fcs_norm=dummy_norm

In [ ]:
%reload_ext autoreload
model =  StackedTransformer(num_blocks=4, n_data_shape=(20, 32, 33, 18), n_heads=8,mlp_mult=4,projection_channels=64)

In [ ]:
#this is where the last run is stored 
checkpoint_path = '/home/jupyter-ayoub/results/Transformerweights/ssrd6/Beste_result/epochs50predictors18CRPSKERNELSTEPlambda0.02k3.3.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, weights_only=False)
model_state_dict = checkpoint['model']
new_state_dict = {k.replace("module.", ""): v for k, v in model_state_dict.items()}
model.load_state_dict(new_state_dict)
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

In [ ]:
#turn the normalized reforecasts into a tensor
fcs_norm_whole= fcs_norm.to_array(dim='variable')
print(fcs_norm_whole.sizes)
#print(fcs_norm_whole)
print(fcs_norm_whole.shape)
rearranged_10= fcs_norm_whole.transpose('time', 'number','step', 'latitude', 'longitude', 'variable')
fcs_whole_tensor=torch.tensor(rearranged_10.values)
print(fcs_whole_tensor.shape) 

In [ ]:
#same for unnormalized data, because I need them for the rescaling 
fcs_array_10 =fcs_10.to_array(dim='variable')
fcs_rearranged_10= fcs_array_10.transpose('time', 'number','step', 'latitude', 'longitude', 'variable')
fcs_whole_notnorm_tensor=torch.tensor(fcs_rearranged_10.values)
print(fcs_whole_notnorm_tensor.shape)

In [ ]:
MBM_one=np.empty((209,11,20,32,33),dtype=float)
for i in range(20):
    mbm1=np.load(f"/home/jupyter-ayoub/ClassicalMBM/resultsClassicalMBM/{target}/MBM_{target}_{i}.npy")
    MBM_one[:,:,i,:,:]=mbm1.squeeze()

In [ ]:
postpro_10_whole = torch.empty((209,11,20,32,33,1))

# Apply model in chunks of 10
with torch.no_grad():
    for i in range(0, 209, 2):
        # Get the end index for this chunk
        end = min(i + 2, 209)  # Ensure we don't go out of bounds

        # Apply the model to the chunk and store in the corresponding location in postpro_10_whole
        #print(fcs_whole_tensor[i:end].shape)
        #print(postpro_10_whole[i:end].shape)
        postpro_10_whole[i:end] = model(fcs_whole_tensor[i:end])

In [ ]:
# Extract ensemble forecasts
ensemble_one = postpro_10_whole[...,0]

# Compute standard deviation & mean across ensemble dimension (dim=1)
scale_std, scale_mean = torch.std_mean(fcs_whole_notnorm_tensor[..., 15], dim=1, unbiased=True)

# Function to scale and clamp ensemble data
def scale_and_clamp(ensemble):
    scaled = ensemble * scale_std.unsqueeze(1) + scale_mean.unsqueeze(1)
    return torch.clamp(scaled, min=1e-6).permute(1, 0, 2, 3, 4)

# Apply scaling and clamping
scaled_ensemble_one = scale_and_clamp(ensemble_one)

In [ ]:
# Define coordinates with member first
coords_forecast = {
    'member': test_rfcs['number'].rename({'number': 'member'}),
    'time': obs['time'],
    'step': obs['step'],
    'latitude': obs['latitude'],
    'longitude': obs['longitude'],
}


# Define dimensions with member first
dims_forecast = ('member', 'time', 'step', 'latitude', 'longitude')

# Convert numpy arrays to xarray DataArray with the specified order
MBM_one=xr.DataArray(np.transpose(MBM_one,(1,0,2,3,4)), coords=coords_forecast, dims=dims_forecast)
full_ensemble_one= xr.DataArray(scaled_ensemble_one, coords=coords_forecast, dims=dims_forecast)

# Compute mean and std across the member dimension
mean_one = full_ensemble_one.mean(dim='member')
std_one = full_ensemble_one.std(dim='member')

# Adjust raw_fcs to have 'member' first
raw_fcs = test_rfcs.rename({'number': 'member'})
raw_fcs = raw_fcs.transpose('member', 'time', 'step', 'latitude', 'longitude')
raw_mean=raw_fcs.mean(dim='member')
raw_std=raw_fcs.std(dim='member')

In [ ]:
#ALL ENSEMBLES
Tformer_one=full_ensemble_one
MBM_one = MBM_one
Raw = raw_fcs['ssrd6']

In [ ]:
def compute_solar_power_from_ssrd6(ssrd6_data, efficiency=0.18, panel_area=1000.0):
    """
    Zet ssrd6 (J/m² over 6 uur) om naar geschat vermogen (W) van een zonne-energiesysteem.
    Werkt op een numpy array of xarray DataArray.

    Parameters:
    - ssrd6_data: ndarray met shape (ensemble, time, lat, lon)
    - efficiency: float, systeemefficiëntie (typisch 0.15–0.20)
    - panel_area: float, totale paneeloppervlakte in m²

    Returns:
    - power_output: zelfde shape als input, vermogen in watt
    """
    irradiance_avg = ssrd6_data / (6 * 3600)  # W/m² gemiddeld over 6 uur
    power_output = efficiency * irradiance_avg * panel_area  # W

    return power_output


In [ ]:
import torch

def solar_irradiance_to_power(ghi_tensor: torch.Tensor, eta_pv: float = 0.18, area_pv: float = 1.0) -> torch.Tensor:
    """
    De paper "Current-Day and Future Dunkelflaute Risks for Belgium", in Section 2 
    "Solar power is computed from GHI data by assuming a fixed conversion efficiency of 18% for the installed PV capacity. 
    The efficiency is applied directly to the GHI to estimate solar power generation."
    
    
    Zet globale horizontale instraling (GHI) om naar opgewekt vermogen van een PV-systeem.
    
    Parameters:
    - ghi_tensor (torch.Tensor): Tensor met GHI-waarden in W/m².
    - eta_pv (float): Rendement van het PV-systeem (standaard 0.18 voor 18%).
    - area_pv (float): Oppervlakte van het PV-systeem in m² (standaard 1.0 m²).
    
    Returns:
    - torch.Tensor: Tensor met opgewekt vermogen in Watt.
    """
    return eta_pv * area_pv * ghi_tensor


In [ ]:
# Stel: Tformer_one.shape = (11, 8, 32, 33)
solar_power_output = solar_irradiance_to_power(
    Tformer_one
)

In [ ]:
# Stel: Tformer_one.shape = (11, 8, 32, 33)
solar_power_output = compute_solar_power_from_ssrd6(
    Tformer_one,
    efficiency=0.19,     # bijvoorbeeld voor moderne panelen
    panel_area=10000.0   # stel dat je 10.000 m² PV-oppervlak modelleert
)

In [ ]:
solar_power_output.shape  # (11, time, lat, lon)

In [ ]:
solar_energy_kWh = solar_power_output * 6 / 1000  # W × h → Wh → kWh

In [ ]:
# Som over alle gridpoints en ensembleleden → totaal vermogen per tijd
total_power_per_time = solar_power_output.sum(axis=(0, 2, 3))  # shape: (time,)

In [ ]:
mean_power_per_time = solar_power_output.mean(axis=0).sum(axis=(1, 2))  # shape: (time,)

In [ ]:
# Plotten
plt.figure(figsize=(10, 5))
plt.plot(mean_power_per_time, marker='o')
plt.title("Gemiddeld voorspeld vermogen per tijdstap (over ensemble en grid)")
plt.xlabel("Tijdstap (6-uurs intervallen)")
plt.ylabel("Vermogen (Watt)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Selecteer enkel de eerste 4 tijdstappen (1 dag = 4 x 6u)
solar_power_day1 = solar_power_output[:, :4, :, :]  # shape: (ensemble, 4, lat, lon)

# Gemiddeld over ensemble en som over grid → vermogen per tijdstap
mean_power_day1 = solar_power_day1.mean(axis=0).sum(axis=(1, 2))  # shape: (4,)

# Plotten
plt.figure(figsize=(8, 5))
plt.plot(mean_power_day1, marker='o', color='orange')
plt.title("Gemiddeld voorspeld vermogen per 6u tijdsblok (dag 1)")
plt.xlabel("Tijdstap (6-uurs intervallen)")
plt.xticks(ticks=range(4), labels=["00–06", "06–12", "12–18", "18–24"])
plt.ylabel("Vermogen (Watt)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
sample = solar_power_output[0]  # shape: (209, 20, 32, 33)

In [ ]:
time_len, ens_len, lat_len, lon_len = sample.shape

random_time = np.random.randint(0, time_len)
random_ens = np.arange(ens_len)
random_lat = np.random.randint(0, lat_len)
random_lon = np.random.randint(0, lon_len)

# Extract ensemblewaarden (vorm: ensemble,)
power_at_point = sample[random_time, :, random_lat, random_lon]

# Plot ensembleverdeling
import matplotlib.pyplot as plt
plt.figure(figsize=(7, 5))
plt.bar(random_ens, power_at_point, color='orange')
plt.title(f"Voorspeld vermogen per ensemblelid\nTijdstap={random_time}, Locatie=({random_lat},{random_lon})")
plt.xlabel("Ensemblelid")
plt.ylabel("Vermogen (Watt)")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# Check shape
print("Sample shape:", sample.shape)
assert sample.ndim == 4, "Verwacht shape (time, ensemble, lat, lon)"

time_len, ens_len, lat_len, lon_len = sample.shape

# Kies een random gridlocatie
random_lat = np.random.randint(0, lat_len)
random_lon = np.random.randint(0, lon_len)

# Neem eerste 4 tijdstappen (één dag)
time_indices = np.arange(4)

# Plot lijn per ensemblelid
import matplotlib.pyplot as plt
plt.figure(figsize=(9, 5))
for ens in range(ens_len):
    plt.plot(time_indices, sample[time_indices, ens, random_lat, random_lon],
             marker='o', alpha=0.6, label=f'Ens {ens}')

plt.title(f"Voorspeld vermogen per ensemblelid over 1 dag\nLocatie=({random_lat},{random_lon})")
plt.xlabel("6-uurs tijdsblok")
plt.xticks(ticks=range(4), labels=["00–06", "06–12", "12–18", "18–24"])
plt.ylabel("Vermogen (Watt)")
plt.grid(True)
plt.tight_layout()
plt.show()